# Test of GraphSAGE
- use DGL
- predict `graphs`
- valid, test data are in the training dataset

In [25]:
import os
import dgl
import csv
import json
import torch
import torch as th
import pandas as pd
import torch.nn as nn
import dgl.nn as dglnn
import torch.nn.functional as F

from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
from torch.optim import AdamW, lr_scheduler
from dgl.nn import GraphConv, GATConv, SAGEConv
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup

- check the GPU and assign the GPU by the best memory usage

In [2]:
def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")
    print("there's no available GPU")

# device = torch.device(f"cuda:{1}")
print(device)


cuda:0


## Fix the seed

In [3]:
#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Load the embedding

In [4]:
DIM = 50

embedding = f'transH_{DIM}'
with open(f"../../data/4_embedding/synthesize/{embedding}.vec.json", "r") as f:
    tmp = json.load(f)

index2entemb = {idx:emb for idx, emb in enumerate(tmp["ent_embeddings.weight"])}
index2relemb = {idx:emb for idx, emb in enumerate(tmp["rel_embeddings.weight"])}

In [5]:
len(index2entemb)

26868

In [6]:
len(index2relemb)

23

In [7]:
with open("../../data/source_data/before_embedding/3.10/all_graph_data.jsonl", "r") as f:
    print("Loading the data...")

    # only process 40000 data from 400000 data
    wanted_data = 399000
    input_data = []
    for idx, line in tqdm(enumerate(f), total=wanted_data, desc="Loading"):
        if idx == wanted_data:
            break
        input_data.append(json.loads(line))
        
    print("FINISH...")

Loading the data...


Loading:   0%|          | 0/399000 [00:00<?, ?it/s]

FINISH...


In [8]:
len(input_data)

399000

- Convert the 'node_feat' and 'edge_attr' from int to embedding
    - takes about 45 min to transform the embedding
    - if use original method -> takes about 60 hours

In [9]:
# If type(input_data[0] == dict)
for data_point in tqdm(input_data):
    data_point['node_feat'] = [index2entemb[node_id] for node_id in data_point['node_feat']]
    data_point['edge_attr'] = [index2relemb[edge_id] for edge_id in data_point['edge_attr']]


# If type(input_data[0] == str)
# for idx, data in tqdm(enumerate(input_data)):
    
#     # make the data from string to int
#     data_point = json.loads(data)

#     data_point['node_feat'] = [index2entemb[node_id] for node_id in data_point['node_feat']]
#     data_point['edge_attr'] = [index2relemb[edge_id] for edge_id in data_point['edge_attr']]

#     input_data[idx] = data_point

  0%|          | 0/399000 [00:00<?, ?it/s]

In [10]:
input_data[1]['node_feat']

[[-0.13400593400001526,
  0.0745830237865448,
  0.035443343222141266,
  0.12708544731140137,
  0.1313164234161377,
  -0.11280117183923721,
  0.03557184711098671,
  0.13569022715091705,
  0.04212702810764313,
  0.015788700431585312,
  -0.12025688588619232,
  -0.03752604499459267,
  0.1454906016588211,
  0.15406882762908936,
  -0.02328800968825817,
  0.04003113880753517,
  0.044135212898254395,
  -0.033310115337371826,
  0.07393254339694977,
  -0.01879263110458851,
  0.06671767681837082,
  -0.054439689964056015,
  -0.09587514400482178,
  0.11176080256700516,
  -0.02801632322371006,
  -0.08088016510009766,
  0.00047018786426633596,
  -0.13521791994571686,
  0.010783975943922997,
  0.022552018985152245,
  0.036946117877960205,
  -0.0462837815284729,
  -0.03688304126262665,
  0.029622813686728477,
  -0.07293407618999481,
  0.06105346232652664,
  -0.00059130834415555,
  -0.11954065412282944,
  -0.11781751364469528,
  -0.051076941192150116,
  0.1166776791214943,
  -0.02639642171561718,
  -0.0

## Data Loader

In [11]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]
        return data

def collate(samples):
    data_list = samples
    batched_graphs = []
    for data in data_list:
        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"])

        g.ndata['feat'] = th.tensor(data["node_feat"])
        g.edata['feat'] = th.tensor(data["edge_attr"])
        g.edata['label'] = th.tensor(data["labels"])  # Add edge labels to graph

        batched_graphs.append(g)
    
    return dgl.batch(batched_graphs)

In [12]:
# split 8:1:1 (train, valid, test)
train_data, test_data = train_test_split(input_data, test_size=0.2, random_state=42)
valid_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)


dataset_data = {
    'train': GraphDataset(train_data, device),
    'valid': GraphDataset(valid_data, device),
    'test': GraphDataset(test_data, device)
}

print("Datasets loaded and ready for training!")

Datasets loaded and ready for training!


In [13]:
len(dataset_data['test'])

39900

In [14]:
dataset_data['train'][0]

{'labels': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


- choose batch size

In [15]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        # do not shuffle the testing dataset
        if dataset_name == "test":
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)    
        else:
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

dataloaders = create_dataloaders(200)

In [16]:
edge_embedding_dim = 0

# Assuming dataloaders is a dictionary with 'test' as one of the keys
for batch in dataloaders['train']:
    print(batch, "\n")
    
#     print("edata:", batch.edata, '\n')
    print("edata['feat'] size:", batch.edata['feat'].shape, '\n')
    print("edata['label']:", batch.edata['label'])

    edge_embedding_dim = batch.edata['feat'].shape[1]

    break  # To break out after the first batch if needed

print("\n\nedge embedding dimension: ", edge_embedding_dim)

Graph(num_nodes=50638, num_edges=103574,
      ndata_schemes={'feat': Scheme(shape=(50,), dtype=torch.float32)}
      edata_schemes={'feat': Scheme(shape=(50,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}) 

edata['feat'] size: torch.Size([103574, 50]) 

edata['label']: tensor([0, 0, 0,  ..., 0, 0, 0])


edge embedding dimension:  50


- Turn the print message to a log file

In [17]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"./log_message/{formatted_time}_GraphSAGE_transH_50-edge.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

./log_message/0124_14:21_GraphSAGE_transH_50-edge.log


### Model

In [18]:
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GraphSAGE, self).__init__()
        self.layer1 = dglnn.SAGEConv(in_dim, hidden_dim, 'pool')
        self.layer2 = dglnn.SAGEConv(hidden_dim, out_dim, 'pool')
#         self.dropout = nn.Dropout(0.25)

    def forward(self, g, inputs):
        h = self.layer1(g, inputs)
        h = torch.relu(h)
#         h = self.dropout(h)
        new_node_feat = self.layer2(g, h)
    
        return new_node_feat

In [19]:
class MLPPredictor(nn.Module):
    def __init__(self, out_feats, out_classes, edge_embedding_dim):
        super().__init__()
        self.W = nn.Linear(out_feats*2 + edge_embedding_dim, out_classes)

    def apply_edges(self, edges, edge_feat):
#     def apply_edges(self, edges):

        h_u = edges.src['new_node_feat']
        h_v = edges.dst['new_node_feat']
        
        num_edges, edge_feat_dim = edge_feat.shape
        
        h_e = edge_feat
        
        score = self.W(torch.cat([h_u, h_v, h_e], 1))

        return {'score': score}


    def forward(self, graph, new_node_feat, edge_feat):
        with graph.local_scope():
            graph.ndata['new_node_feat'] = new_node_feat
#             graph.apply_edges(self.apply_edges)

            # 在 apply_edges 时传递 edge_feat
            graph.apply_edges(lambda edges: self.apply_edges(edges, edge_feat))
            return graph.edata['score']

In [20]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_classes, edge_embedding_dim):
        super().__init__()
        self.sage = GraphSAGE(in_features, hidden_features, out_features)
        self.pred = MLPPredictor(out_features, num_classes, edge_embedding_dim)
      
    def forward(self, g, node_feat, edge_feat, return_logits=False):
        new_node_feat = self.sage(g, node_feat)
        logits = self.pred(g, new_node_feat, edge_feat)
        
        return logits

- Model Forward  

In [21]:
def model_fn(batched_g, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
#     batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = batched_g.edata['label'].to(device)
    
#     logits = model(batched_g, batched_g.ndata['feat'].float())
    logits = model(batched_g, batched_g.ndata['feat'].float(), batched_g.edata['feat'].float())

    loss = criterion(logits, labels)

    output = torch.softmax(logits, dim=1)
    preds = output.argmax(1)
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
        
    return loss, accuracy, preds

### Training Loop

- For release the GPU memory
    - no need to restart the kernel

In [22]:
# # For release the GPU memory
# # No need to restart the kernel

# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [23]:
seed = 8787
in_dim = DIM # dimension of the node feature
hidden_dim = 64
out_dim = 128
num_classes = 2 # for DARPA
edge_dim = edge_embedding_dim

lr = 5e-4

total_steps = 100
patience = 5
waiting = 0

In [26]:
model = Model(in_dim, hidden_dim, out_dim, num_classes, edge_dim)
best_model_path = "./checkpoint_graphSAGE/best_model_GraphSAGE_transH_50-plusedge.pt"

optimizer = AdamW(model.parameters(), lr)

scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)

criterion = nn.CrossEntropyLoss()
# criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
same_seeds(seed)
model = model.to(device)
best_val_loss = float('inf')


# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    for batched_g in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
        num_batches += 1
        loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

    scheduler.step()
    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for batched_g in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
            
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)
            add_log_msg("Find a better model!!")

        torch.save(model.state_dict(), best_model_path)
 
    else:
        waiting += 1
        if waiting >= patience:
            add_log_msg("============================== Early stopping ==================================")
            break

  0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/1596 [00:00<?, ?it/s]

01/24/2024, 14:36:30# total batches: 1596
01/24/2024, 14:36:30# Epoch 0 | Train Loss: 0.0909 | Train Accuracy: 0.9709


Validation:   0%|          | 0/200 [00:00<?, ?it/s]

01/24/2024, 14:38:07# Validation Loss: 0.0520 | Validation Accuracy: 0.9794



...

Training:   0%|          | 0/1596 [00:00<?, ?it/s]

01/24/2024, 20:36:31# total batches: 1596
01/24/2024, 20:36:31# Epoch 20 | Train Loss: 0.0241 | Train Accuracy: 0.9866


Validation:   0%|          | 0/200 [00:00<?, ?it/s]

01/24/2024, 20:38:20# Validation Loss: 0.0241 | Validation Accuracy: 0.9867



Training:   0%|          | 0/1596 [00:00<?, ?it/s]

### Testing Loop

In [ ]:
# load the pretrained model
model.load_state_dict(torch.load(best_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batched_g in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):
#         print(f"data:{data[1]}")
        loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, count, which_type='test')
        labels = batched_g.edata['label'].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')

In [ ]:
report_data = classification_report(true_labels, predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

report_df

### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [68]:
seed = 8787
same_seeds(seed)

model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
torch.save(model.state_dict(), 'model3_initial(graphsage)/initial_weight.pth')

In [69]:
# model.layer1.fc_self.weight
model.sage.layer1.fc_self.weight

Parameter containing:
tensor([[ 0.0181, -0.0857,  0.1973,  ...,  0.2417,  0.2702, -0.3041],
        [-0.0768, -0.2723, -0.2001,  ...,  0.2989, -0.1387, -0.1940],
        [ 0.2582, -0.0822,  0.3086,  ..., -0.0257, -0.1119, -0.0335],
        ...,
        [ 0.2274, -0.0411, -0.0334,  ..., -0.1679,  0.2455,  0.2424],
        [ 0.1375,  0.2813,  0.0775,  ...,  0.1337,  0.2065,  0.2618],
        [-0.0951,  0.1010, -0.2586,  ..., -0.1242, -0.0631,  0.0924]],
       requires_grad=True)

- Check if model really load the model_dict

In [70]:
model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
model.sage.layer1.fc_self.weight

Parameter containing:
tensor([[ 0.0181, -0.0857,  0.1973,  ...,  0.2417,  0.2702, -0.3041],
        [-0.0768, -0.2723, -0.2001,  ...,  0.2989, -0.1387, -0.1940],
        [ 0.2582, -0.0822,  0.3086,  ..., -0.0257, -0.1119, -0.0335],
        ...,
        [ 0.2274, -0.0411, -0.0334,  ..., -0.1679,  0.2455,  0.2424],
        [ 0.1375,  0.2813,  0.0775,  ...,  0.1337,  0.2065,  0.2618],
        [-0.0951,  0.1010, -0.2586,  ..., -0.1242, -0.0631,  0.0924]],
       requires_grad=True)

In [ ]:
mapping_file = '../data/label2id.txt'
label_mapping = {}
with open(mapping_file, 'r') as f:
    next(f)
    for line in f:
        parts = line.strip().split(' ')
        label_mapping[int(parts[1])] = parts[0]
        
# 将映射后的标签应用到true和predicted标签列表
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# 生成Scikit-learn报告信息的DataFrame
report_data = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

# mapped_true_labels_np = np.array(mapped_true_labels)
# mapped_predicted_labels_np = np.array(mapped_predicted_labels)

# print("mapped_true_labels 的形状:", mapped_true_labels_np.shape)
# print("mapped_predicted_labels 的形状:", mapped_predicted_labels_np.shape)

report_folder = 'classification_report'
os.makedirs(report_folder, exist_ok=True)

count = 0
while True:
    report_filename = f'classification_report-transE_50-graphSAGE-{count}.xlsx'
    labels_filename = f'mapped_true_predicted_labels-transE_50-graphSAGE-{count}.xlsx'
    
    report_path = os.path.join(report_folder, report_filename)
    labels_path = os.path.join(report_folder, labels_filename)
    
    if not os.path.exists(report_path) and not os.path.exists(labels_path):
        break
    count += 1

    
report_df.to_excel(report_path, index_label='Label')

mapped_labels_df = pd.DataFrame({'true_label': mapped_true_labels, 'predicted_label': mapped_predicted_labels})
mapped_labels_df.to_excel(labels_path, index=False)

add_log_msg(f"report path: {report_path}")
add_log_msg(f"label path: {labels_path}")

mapped_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"mapped_report:\n{mapped_report}")

In [ ]:
# report_data = classification_report(true_labels, predicted_labels, output_dict=True)
# report_df = pd.DataFrame(report_data).transpose()

# report_folder = 'classification_report'
# os.makedirs(report_folder, exist_ok=True)

# count = 0
# while True:
#     report_filename = f'classification_report-transE_50-graphSAGE-{count}.xlsx'
#     labels_filename = f'mapped_true_predicted_labels-transE_50-graphSAGE-{count}.xlsx'
    
#     report_path = os.path.join(report_folder, report_filename)
#     labels_path = os.path.join(report_folder, labels_filename)
    
#     if not os.path.exists(report_path) and not os.path.exists(labels_path):
#         break
#     count += 1

    
# report_df.to_excel(report_path, index_label='Label')

# labels_df = pd.DataFrame({'true_label': true_labels, 'predicted_label': predicted_labels})
# labels_df.to_excel(labels_path, index=False)

# add_log_msg(f"report path: {report_path}")
# add_log_msg(f"label path: {labels_path}")

# report = classification_report(true_labels, predicted_labels)
# add_log_msg(f"report:\n{report}")